In [1]:
import numpy as np
import tensorflow as tf 
import tflearn
from collections import deque
import random
import pdb
from replay_memory import ReplayMemory
#from AC_networks import ActorNet, CriticNet
from AC_networks_2 import AC_Net
from test_game import test_env
import os

In [2]:
class input_struct:
    def __init__(self, net):
        self.net = net


In [3]:
import gym
game = gym.make('LunarLander-v2')
#game = gym.make('CartPole-v0')
state_dim = game.observation_space.shape[0]
action_dim = game.action_space.n
#for i_episode in range(20):
#    observation = env.reset()
#    for t in range(100):
#        env.render()
#        print(observation)
#        action = env.action_space.sample()
#        observation, reward, done, info = env.step(action)
#        if done:
#            print("Episode finished after {} timesteps".format(t+1))
#            break

[2016-12-26 23:07:19,513] Making new env: LunarLander-v2


In [4]:
# info = graph, dim_s, dim_a, hid_layers, learning_rate, tau
MAX_EPISODE = 10000
MAX_STEP = 1000
#MINIBATCH_SIZE = 16
GAMMA = 0.99
LEARNING_RATE = 1e-4 #7e-5 lr/2
ENTROPY_BETA = 0
#buff = ReplayMemory(500)
info = input_struct('actor')
try:
    actor
except:
    pass
else:
    AC_src.reset_net() 
    
graph = tf.Graph()
with graph.as_default():
    info.graph = graph
    info.dim_s = state_dim
    info.dim_a = action_dim
    info.shared_layers = np.array([64])
    info.act_hid_layers = np.array([32])
    info.crit_hid_layers = np.array([32])
    info.learning_rate = LEARNING_RATE
    info.entropy_beta = ENTROPY_BETA
    thread = -1
    AC_src = AC_Net(info, thread)
    thread = 1
    AC_dst = AC_Net(info, thread)
    

In [8]:
#game = test_env()
print_ep = 100
save_ep = 1000
lr_decay_ep = 100
lr = LEARNING_RATE
#buff.clear()
last_ep = 0
start_update = 0
with tf.Session(graph=graph) as sess:
    AC_src.set_up(sess)
    AC_dst.set_up(sess)
    sess.run(tf.initialize_all_variables())
    
    #saver = tf.train.Saver()
    #filename = './saved_nets - last/Episod_4999.chk'
    #saver.restore(sess, filename)
    #pdb.set_trace()
    ep_reward = np.zeros(MAX_EPISODE)
    ep_success = np.zeros(MAX_EPISODE)
    ep_crash = np.zeros(MAX_EPISODE)
    avg_var = 0
    
    for ep in xrange(MAX_EPISODE):
        s = game.reset()
        var = 0
        #ep_reward[ep] = 0
        #print(ep)
        for st in xrange(MAX_STEP):
            A = AC_src.take_action(s.reshape(-1,info.dim_s))
            a = np.argmax(A)
            s2, r, terminal, report = game.step(a) # A changes to a
            
            #print(terminal, r)
            #s2, r = game.step(a)
            #terminal = 1
            #print('s: %d,  a: %d,   s2: %d,   r: %d' %(s, np.argmax(A), s2, r))
            
            #buff.add(np.reshape(s, (actor.dim_s,)).astype(np.float32), np.reshape(A, (actor.dim_a,)).astype(np.float32),\
            #         np.array(r).astype(np.float32), np.reshape(s2, (actor.dim_s,)).astype(np.float32),\
            #         np.reshape(terminal, (1,)))

            #pdb.set_trace()
            #if buff.size() > MINIBATCH_SIZE:
            if True:
                (s_batch, a_batch, r_batch, s2_batch, t_batch) = (s, A, r, s2, terminal)#\
                    #buff.sample_batch(MINIBATCH_SIZE)
                start_update = 1
                #pdb.set_trace()
                s_batch = np.array(s_batch).reshape(-1, info.dim_s)
                s2_batch = np.array(s2_batch).reshape(-1, info.dim_s)
                a_batch = np.array(a_batch).reshape(-1, info.dim_a)
                r_batch = np.array(r_batch).reshape(-1, 1)
                t_batch = np.array(t_batch).reshape(-1, 1)
                
                
                #Q_2 = critic.predict_target(s2_batch)
                Q_2 = AC_src.predict(s2_batch)

                R_batch = r_batch + GAMMA * Q_2 * (1-t_batch)
                #if((ep % print_ep) == (print_ep-1)) and (st == 0):
                    #pdb.set_trace()
                    
                predicted_Q, _ = AC_src.train(s_batch, a_batch, np.reshape(R_batch, (1, 1)), lr)
            
                #td_batch = R_batch - predicted_Q
                #pdb.set_trace()
                #actor.train(s_batch, a_batch, np.reshape(td_batch, (MINIBATCH_SIZE, 1)), lr)
            
            ep_reward[ep] += (1.*r)
            #pdb.set_trace()
            var += np.sum(AC_src.give_policy(s.reshape(-1,info.dim_s))**2)
            s = s2
            if terminal:
                var = var/st
                avg_var += (var/print_ep)
                break
        if terminal == False:
            var = var/MAX_STEP
            avg_var += (var/print_ep)
        if (r >= 100):
            ep_success[ep] = 1
        elif (r <= -100):
            ep_crash[ep] = 1
        #if(start_update):
        #    actor.update_target_network
        #    critic.update_target_network
        
        if ((ep % print_ep) == (print_ep-1)) or (ep == 0):
            print('Episode: %d --- Ave R: %.1f  |  Success: %.2f  |  crash: %.2f  |  Max R: %.1f  |  var: %.2f  |  lr:%.2f*1e-4'\
                  %(ep, np.mean(ep_reward[last_ep:ep+1]), np.mean(ep_success[last_ep:ep+1]), \
                    np.mean(ep_crash[last_ep:ep+1]), np.amax(ep_reward[last_ep:ep+1]), var, 1e4*lr))
            last_ep = ep
            avg_var = 0
            #pdb.set_trace()
        if (ep % save_ep) == (save_ep-1):
            print(' ***** Saving...  at episode %d ******' %ep)
            saver = tf.train.Saver()
            filename = './saved_nets/Episod_' + str(ep) + '.chk'
            try:
                saver.save(sess, filename)
            except:
                os.mkdir('./saved_nets')  
            with open('./saved_nets/rewards_file.txt', 'w') as f:
                for i in xrange(MAX_EPISODE):
                    f.write('%f\n' %ep_reward[i])
        if (ep % lr_decay_ep) == (lr_decay_ep-1):
            lr = 0.97 * lr
    '''
    pdb.set_trace()
    AC_dst.sync_from_source(AC_src)
    ep_dst = 0
    for st in xrange(MAX_STEP):
        A = AC_src.take_action(s.reshape(-1,info.dim_s))
        a = np.argmax(A)
        s2, r, terminal, report = game.step(a)
        ep_dst += 1. * r
        s = s2
    print('dest_reward: %f' %(ep_dst))
    '''
    
print('DONE') 
    

Episode: 0 --- Ave R: -544.4  |  Success: 0.00  |  crash: 1.00  |  Max R: -544.4  |  var: 0.25  |  lr:1.00*1e-4
Episode: 99 --- Ave R: -254.7  |  Success: 0.00  |  crash: 0.99  |  Max R: -26.1  |  var: 0.29  |  lr:1.00*1e-4
Episode: 199 --- Ave R: -352.0  |  Success: 0.00  |  crash: 1.00  |  Max R: -56.2  |  var: 0.40  |  lr:0.97*1e-4
Episode: 299 --- Ave R: -238.0  |  Success: 0.00  |  crash: 0.94  |  Max R: -31.6  |  var: 0.36  |  lr:0.94*1e-4
Episode: 399 --- Ave R: -186.8  |  Success: 0.00  |  crash: 0.88  |  Max R: -11.6  |  var: 0.50  |  lr:0.91*1e-4
Episode: 499 --- Ave R: -129.6  |  Success: 0.01  |  crash: 0.75  |  Max R: 26.6  |  var: 0.46  |  lr:0.89*1e-4
Episode: 599 --- Ave R: -62.5  |  Success: 0.09  |  crash: 0.49  |  Max R: 138.8  |  var: 0.49  |  lr:0.86*1e-4
Episode: 699 --- Ave R: -184.1  |  Success: 0.02  |  crash: 0.98  |  Max R: 158.7  |  var: 0.51  |  lr:0.83*1e-4
Episode: 799 --- Ave R: -235.7  |  Success: 0.00  |  crash: 1.00  |  Max R: -169.3  |  var: 0.59  | 

Type is unsupported, or the types of the items don't match field type in CollectionDef.
'NoneType' object has no attribute 'name'
[2016-12-26 23:18:31,749] Error encountered when serializing data_augmentation.
Type is unsupported, or the types of the items don't match field type in CollectionDef.
'NoneType' object has no attribute 'name'
Type is unsupported, or the types of the items don't match field type in CollectionDef.
'NoneType' object has no attribute 'name'
[2016-12-26 23:18:31,754] Error encountered when serializing data_preprocessing.
Type is unsupported, or the types of the items don't match field type in CollectionDef.
'NoneType' object has no attribute 'name'


Episode: 1099 --- Ave R: -174.3  |  Success: 0.00  |  crash: 0.96  |  Max R: -45.4  |  var: 0.50  |  lr:0.74*1e-4
Episode: 1199 --- Ave R: -180.0  |  Success: 0.00  |  crash: 0.92  |  Max R: -60.2  |  var: 0.45  |  lr:0.72*1e-4
Episode: 1299 --- Ave R: -197.3  |  Success: 0.00  |  crash: 0.85  |  Max R: -15.3  |  var: 0.52  |  lr:0.69*1e-4
Episode: 1399 --- Ave R: -224.9  |  Success: 0.00  |  crash: 0.96  |  Max R: -144.8  |  var: 0.50  |  lr:0.67*1e-4
Episode: 1499 --- Ave R: -245.0  |  Success: 0.00  |  crash: 0.83  |  Max R: -126.5  |  var: 0.51  |  lr:0.65*1e-4


KeyboardInterrupt: 

In [ ]:
EP = MAX_EPISODE - 1
with tf.Session(graph=graph) as sess:
    
    actor.set_up(sess)
    critic.set_up(sess)
    sess.run(tf.initialize_all_variables())
    saver = tf.train.Saver()
    filename = './saved_nets/Episod_' + str(EP) + '.chk'
    saver.restore(sess, filename)
    
    ep_reward = 0
    s = game.reset()
    for st in xrange(MAX_STEP):
        game.render()
        print(actor.give_policy(s.reshape(-1,info.dim_s)))
        A = actor.take_action(s.reshape(-1,info.dim_s))
        a = np.argmax(A)
        s2, r, terminal, report = game.step(a)
        s = s2
        ep_reward += (1.*r)
        if terminal:
            break
    print(r)
    print('reward:%f' %np.mean(ep_reward))

In [ ]:
with open('rewards_file.txt', 'w') as f:
    for i in xrange(MAX_EPISODE):
        f.write('%f\n' %ep_reward[i])


In [ ]:
np.random.choice(2, 1)